# 术语及定义

* 顶点：又称节点，是图的基础部分。它可以有自己的名字，我们称作“键”，顶点也可以带有附加信息，我们称作“有效载荷”
* 边：图的另一基础部分，两个顶点通过一条边连接，表示他们之间存在关系。边既可以是单向[有向图]也可以是双向的。
* 权重：边可以带权重，用来表示从一个顶点到另一个顶点的成本

# 图的抽象数据类型

* Graph():新建一个空图
* addVertex(vert):向图中添加一个顶点实例
* addEdge(fromVert, toVert):向图中添加一条有向边，用于连接顶点fromVert和toVert
* addEdge(fromVert, toVert, weight):向图中添加一条带有权重weight的有向边，用于连接fromVert和toVert
* getVertex(vertKey):在图中找到名为vertKey的顶点
* geVertices():以列表的形式返回图中所有顶点
* in:通过 vertex in graph 这样的语句，在顶点存在时返回True 否则返回False

In [3]:
# 简单的带权有向图

In [4]:
# Vertex
class Vertex:
    distance = None
    pred = None
    color = None
    def __init__(self, key):
        self.id = key
        self.connectedTo = {}
        
        
    def addNeighbor(self, nbr, weight=0):   # neighbor 邻接？
        """
            添加相邻的顶点
        :param nbr: 
        :param weight: 
        :return: 
        """
        self.connectedTo[nbr] = weight
    
    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])
    
    def setDistance(self, distance):
        self.distance = distance
    
    def setPred(self, pred):
        self.pred = pred
    
    def setColor(self, color):
        self.color = color
    
    def getDistance(self):
        return self.distance
    
    def getPred(self):
        return  self.pred
    
    def getColor(self):
        return self.color
    
    def getConnections(self):
        """
            返回邻接表中所有顶点
        :return: 
        """
        return self.connectedTo.keys()
    
    def getId(self):
        return self.id
    
    def getWeight(self, nbr):
        """
            返回从当前顶点到参数传入顶点之间的边的权重
        :param nbr: 
        :return: 
        """
        return self.connectedTo[nbr]

In [5]:
# Graph
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self, key):
        self.numVertices += 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self, n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self, item):
        return item in self.vertList

    def addEdge(self, f, t, cost=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(f)
            
        self.vertList[f].addNeighbor(self.vertList[t], cost)

    def getVertices(self):  # 返回的是节点名称而不是节点对象
        return self.vertList.keys()

    def __iter__(self): # 节点对象通过遍历Graph类对象获取
        return iter(self.vertList.values())
    
    

##### 示例

使用Graph类和Vertex类创建一个邻接矩阵图

* 首先创建6个顶点，依次编号0~5.
* 然后打印字典。 注意：对于每个键，我们创建了一个Vertex实例
* 接着，添加顶点连接起来的边
* 最后用一个嵌套循环验证图中每一条边有已经被正确存储

In [6]:
g = Graph()

In [7]:
for i in range(6):  # 向图中添加六个顶点
    g.addVertex(i)

In [8]:
g.vertList

{0: <__main__.Vertex at 0x2538d13f2d0>,
 1: <__main__.Vertex at 0x2538d9c8950>,
 2: <__main__.Vertex at 0x2538d9c8250>,
 3: <__main__.Vertex at 0x2538d9c8990>,
 4: <__main__.Vertex at 0x2538da96850>,
 5: <__main__.Vertex at 0x2538d9e4110>}

In [9]:
# 添加边
# 这里自定义连个顶点之间是相邻的，边的权重是多少
# 获取有什么标准规则，但是现在我并不清楚
g.addEdge(0, 1, 5)
g.addEdge(0, 5, 2)
g.addEdge(1, 2, 4)
g.addEdge(2, 3, 9)
g.addEdge(3, 4, 7)
g.addEdge(3, 5, 3)
g.addEdge(4, 0, 1)
g.addEdge(5, 4, 8)
g.addEdge(5, 2, 1)

In [10]:
for v in g: # 遍历图中的所有顶点
    for w in v.getConnections():
        # 遍历当前顶点的连接信息，即：查找当前顶点的所有相邻顶点
        # 输出当前顶点及其相邻顶点的信息
        # 这里的图是单向的，所以对照课本图7-2时，v0连接的只有v1和v5，
        # 对于图，两个顶点之间是否有边连接
        # 边具有方向，v0连接v1和v5，所以v0与v1和v5相邻，但是v1和v5不相邻
        print("(%s, %s)" % (v.getId(), w.getId()))

(0, 1)
(0, 5)
(1, 2)
(2, 3)
(3, 4)
(3, 5)
(4, 0)
(5, 4)
(5, 2)


# 广度优先搜索


### 词题图
    将单次fool转换成sage
    在解决词梯问题时，必须每次只替换一个字母，并且每一步的结果都必须是一个单词，而不能是不存在的词
    
词梯问题的变体
* 给定步数内完成转换
* 必须用到某个单词

学习怎样从起始单词转换到结束单词的最小步数
* 用图表示单词之间的关系
* 广度优先搜索的图算法找到从起始单词到结束单词的最短路径

### 构建词题图

* 如何用图来表示大的单次集合：两个单词之间的区别仅在于有一个不同的字母，则用一条边连接
    * 如果创建这样一个图，那么其中任意一条连接两个单词的路径就是词梯问题的一个解

In [11]:
# 词梯图
wordList = ['fool', 'pool', 'poll', 'pole', 'pale', 'sale', 'sage']
wordGraph = Graph()

In [12]:
for vertex in wordList:
    wordGraph.addVertex(vertex)

In [13]:
d = {}
for word in wordList:
    for i in range(len(word)):
        becket = word[:i] + "_" + word[i+1:]
        if becket in d:
            d[becket].append(word)
        else:
            d[becket] = [word]

for bucket in d.keys():
    for word1 in d[bucket]:
        for word2 in d[bucket]:
            if word1 != word2:
                wordGraph.addEdge(word1, word2)

In [14]:
for v in wordGraph:
    for w in v.getConnections():
        print("(%s, %s)" % (v.getId(), w.getId()))

(fool, pool)
(pool, fool)
(pool, poll)
(poll, pool)
(poll, pole)
(pole, poll)
(pole, pale)
(pale, pole)
(pale, sale)
(sale, pale)
(sale, sage)
(sage, sale)


In [15]:
wordGraph.vertList

{'fool': <__main__.Vertex at 0x2538d113f10>,
 'pool': <__main__.Vertex at 0x2538d12ddd0>,
 'poll': <__main__.Vertex at 0x2538db29510>,
 'pole': <__main__.Vertex at 0x2538db2ac50>,
 'pale': <__main__.Vertex at 0x2538db2b110>,
 'sale': <__main__.Vertex at 0x2538db2b2d0>,
 'sage': <__main__.Vertex at 0x2538db2b290>}

In [52]:
def buildGraph(wordFile):
    """
    
    :param wordFile: 文件路径
    :return: 
    """
    ds = {}
    gs = Graph()
    
    
    # 创建词桶
    wFile = open('word.txt', 'r')
    for line in wFile:
        word = line[:-1]    # 一行一个单词line[:-1]
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            """
            FOOL
            _OOL
            F_OL
            FO_L
            FOO_
            FOOL
            """
            # 通过单词FOOL创建五个标签，作为字典的键，字典的值是一个列表，初始化将FOOL加入列表
            # 然后遍历其他单词，创建创建标签，如果标签在字典中存在，则将该单词加入相应的列表，否则创建新的键
            if bucket in ds:
                ds[bucket].append(word)
            else:
                ds[bucket] = [word]
    # 为同一个桶中的单词添加顶点和边
    # 遍历词桶，桶中每一个键对应的列表中的单词都是相邻的
    for bucket in ds:
        for word1 in ds[bucket]:
            for word2 in ds[bucket]:
                if word1 != word2:
                    gs.addEdge(word1, word2)
    return gs
# 将每一个对象按照特定规则拆分出多个属性
# 感觉有点像网络，这种要怎么描述？？？

In [53]:
# with open('.word.txt', 'w') as f:
#     for word in wordList:
#         f.write(f"{word}\n")

In [54]:
gs = buildGraph(1111)

KeyError: 'pool'

### 广度优先搜索实现

In [ ]:
from functools import wraps


class Queue:
    def __init__(self):
        self.__vals = []
        self.__size = 0

    @staticmethod
    def __addSize(func):
        def wrapper(self, val):
            func(val)
            self.__size += 1

        return wrapper

    @staticmethod
    def __subSize(func):
        def wrapper(self):
            self.__size -= 1
            return func()

        return wrapper

    @__addSize
    def enqueue(self, val):
        self.__vals.append(val)

    @__subSize
    def dequeue(self):
        return self.__vals.pop(0)

    def isEmpty(self):
        return self.__vals == []

    def __len__(self):
        return self.__size

    def __iter__(self):
        return iter(self.__vals)

    def __contains__(self, item):
        return item in self.__vals

In [ ]:
# BFS算法
def bfs(g, start):
    start.setDistance(0)
    start.setPred(None)
    vertQueue = Queue()
    vertQueue.enqueue(start)
    while len(vertQueue) > 0:
        currentVert = vertQueue.dequeue()
        for nbr in currentVert.getConnections():
            if nbr.getColor() == 'white':
                nbr.setColor('gray')
                nbr.setDistance(currentVert.getDistance() + 1)
                nbr.setPred(currentVert)
                vertQueue.enqueue(nbr)
        currentVert.setColor('black')
                

BFS执行流程：
从起点s算起，将它标记为灰色，以表示正在访问它。另外两个变量，distance和predecessor，分别初始化为0和None。
随后，start被放入Queue中，系统化地访问位于队列头部的顶点。
通过遍历邻接表来访问新的顶点，在访问每一个新顶点时，都会检查他的颜色，如果是白色说明没有被访问过则：

     
* 将新的未访问顶点nbr标记为灰色
* 将nbr的predecessor设置成顶点的currentVert
* 将nbr添加到队列尾部。为之后访问该顶点做准备，但是要等currentVert邻接表中的所有其他订单都被访问后才能访问该顶点